In [11]:
import pandas as pd
import requests

url = 'https://data.cms.gov/resource/ehrv-m9r6.json?'

resp = requests.get(url)
# Confirm request was valid:
print(resp)

#Return a dictionary containing JSON parsed into Python objects:
data = resp.json()

# Column headers: First row of data
print(data[0])

# Generate a DF
drg_providers = pd.DataFrame(data, columns=data[0])

print(drg_providers)

<Response [200]>
{u'provider_id': u'10001', u'average_medicare_payments_2': u'4763.73', u'provider_city': u'DOTHAN', u'average_medicare_payments': u'5777.24', u'provider_zip_code': u'36301', u'average_covered_charges': u'32963.07', u'provider_street_address': u'1108 ROSS CLARK CIRCLE', u'hospital_referral_region_description': u'AL - Dothan', u'provider_state': u'AL', u'provider_name': u'SOUTHEAST ALABAMA MEDICAL CENTER', u'drg_definition': u'039 - EXTRACRANIAL PROCEDURES W/O CC/MCC', u'total_discharges': u'91'}
    provider_id average_medicare_payments_2    provider_city  \
0         10001                     4763.73           DOTHAN   
1         10005                     4976.71             BOAZ   
2         10006                     4453.79         FLORENCE   
3         10011                     4129.16       BIRMINGHAM   
4         10016                     4851.44        ALABASTER   
5         10023                     5374.14       MONTGOMERY   
6         10029                    